<a href="https://colab.research.google.com/github/Andron00e/CBM_Optim/blob/main/llama70Bprompts.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# !pip install git+https://github.com/huggingface/peft.git git+https://github.com/huggingface/transformers.git
# !pip install accelerate
# !pip install bitsandbytes
# !pip install sentencepiece
# !pip install datasets


In [1]:
import warnings
warnings.filterwarnings('ignore')

import os
os.environ["CUDA_VISIBLE_DEVICES"]="3"

import auto_gptq
import torch
import json
import os

import bitsandbytes as bnb

from tqdm import tqdm
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

In [ ]:
print('CUDA GPU available: ', torch.cuda.is_available())

Llama-2-70B-chat-GPTQ

In [ ]:
num_gpus = torch.cuda.device_count()

# Iterate over the available GPUs
for i in range(num_gpus):
    # Get the properties of the GPU
    properties = torch.cuda.get_device_properties(i)
    
    # Print the name and memory consumption of the GPU
    print(f"GPU {i}: {properties.name}, Memory: {properties.total_memory}MB")
device = torch.device("cuda:0")
device

In [8]:
model_path = "TheBloke/Llama-2-70B-chat-GPTQ"
# tutorial: https://huggingface.co/TheBloke/Llama-2-70B-chat-GPTQ
# To use a different branch, change revision
# For example: revision="gptq-4bit-32g-actorder_True"
model = AutoModelForCausalLM.from_pretrained(
    model_path,
    torch_dtype = torch.float16,
    device_map = "auto",
    revision="gptq-4bit-32g-actorder_True",
)

tokenizer = AutoTokenizer.from_pretrained(model_path)
tokenizer.pad_token_id = (
    0
)
tokenizer.padding_side = "left"

print(model)

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 8192, padding_idx=0)
    (layers): ModuleList(
      (0-79): 80 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (rotary_emb): LlamaRotaryEmbedding()
          (k_proj): QuantLinear()
          (o_proj): QuantLinear()
          (q_proj): QuantLinear()
          (v_proj): QuantLinear()
        )
        (mlp): LlamaMLP(
          (act_fn): SiLUActivation()
          (down_proj): QuantLinear()
          (gate_proj): QuantLinear()
          (up_proj): QuantLinear()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): LlamaRMSNorm()
  )
  (lm_head): Linear(in_features=8192, out_features=32000, bias=False)
)


In [10]:
pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.float16,
    device_map="auto",
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
    max_length=50
)

In [11]:
def generation_function(prompt: str):
  features = pipeline(prompt)[0]["generated_text"][len(prompt):]
  features = features.split("\n-")
  features = [feat.replace("\n", "") for feat in features]
  features = [feat.strip()for feat in features]
  features = [feat for feat in features if len(feat)>0]
  features = set(features)
  return features

In [22]:
generated_answers = {}
output_file = "data/concepts_llama.json"
# XXX: all_concepts or all_classes?
save_classes = "data/all_concepts.txt"
with open(save_classes, "r") as f:
  classes = f.read().lower().split("\n")

# NOTE: change as needed to all classes
for label in tqdm(classes[:20]):
  generated_answers[label] = {}

  prompt1 = f"List the most important features for recognizing something as a {label}? Write them one by one."
  generated1 = generation_function(prompt1)
  generated_answers[label]["A1"] = list(generated1)

  prompt2 = f"List the things most commonly seen around a {label}. Write them one by one."
  generated2 = generation_function(prompt2)
  generated_answers[label]["A2"] = list(generated2)

  prompt3 = f"Give a generalization for the word {label}"
  generated3 = generation_function(prompt3)
  generated_answers[label]["A3"] = list(generated3)


with open(output_file, "w") as json_file:
    json.dump(generated_answers, json_file, indent=4)

100%|██████████| 20/20 [03:09<00:00,  9.45s/it]


TheBloke/llama-2-70b-Guanaco-QLoRA-fp16

**System:**
{prompt}

**Assistant:**

TheBloke/Llama-2-70B-GGML


**System:**
{system_message}

**User:**
{prompt}

**Assistant:**
